In [1]:
import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
%matplotlib widget
from util import get_path, get_dates_datetime
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi, meijering
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from time import sleep
from skimage.feature import hessian_matrix_det
from experiment_class_surf import Experiment,clean_exp_with_hyphaes
from hyphae_id_surf import clean_and_relabel, get_mother, save_hyphaes, resolve_ambiguity_two_ends,solve_degree4, clean_obvious_fake_tips, get_pixel_growth_and_new_children
from subprocess import call
import open3d as o3d
import sklearn

In [2]:
from directory import directory
# directory = "/scratch/shared/mrozemul/Fiji.app/"

exp_inst = [(40,0,19),(13,0,19),(3,0,19)]
experiments = []
for inst in exp_inst:
    plate = inst[0]
    begin = inst[1]
    end = inst[2]
    dates_datetime = get_dates_datetime(directory,plate)
    print('begin =',dates_datetime[begin],'\n  end =',dates_datetime[end])
    dates_datetime_chosen=dates_datetime[begin:end+1]
    dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
    exp = pickle.load( open(f'{directory}/Analysis_Plate{plate}_{dates[0]}_{dates[-1]}/experiment_{plate}.pick', "rb" ) )
    experiments.append(exp)

begin = 2020-11-19 20:21:00 
  end = 2020-11-23 00:21:00
begin = 2020-07-01 15:57:00 
  end = 2020-07-04 19:57:00
begin = 2020-08-14 00:57:00 
  end = 2020-08-17 04:57:00


In [3]:
exp40 = experiments[0]
exp13 = experiments[1]
exp3 = experiments[2]

In [58]:
plt.close('all')
exp40.plot_raw(19)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
def get_time(exp,t,tp1):
    return(tp1-t)

In [58]:
t = 0
tp1 = 1
exp = exp40
exp.dates[0],exp.dates[1]

('1119_2021', '1120_0021')

In [25]:
exp = exp40
select_hyph = {}
for hyph in exp.hyphaes:
    select_hyph[hyph] = []
    for i,t in enumerate(hyph.ts[:-1]):
        pixels,nodes = get_pixel_growth_and_new_children(hyph,t,hyph.ts[i+1])
        length = np.sum([len(seg) for seg in pixels])
        select_hyph[hyph].append((t,hyph.ts[i+1],length,pixels))
        

In [27]:
max_growths = []
lengths = []
branch_frequ = []
hyph_l = []
for hyph in exp.hyphaes:
    growths = [c[2] for c in select_hyph[hyph]]
    if len(growths)>0:
        length = hyph.get_length_um(hyph.ts[-1])
        nodes = hyph.get_nodes_within(hyph.ts[-1])[0]
        max_growth = np.max(growths)
        lengths.append(length)
        max_growths.append(max_growth)
        branch_frequ.append((len(nodes)-1)/(length+1))
        hyph_l.append(hyph)

In [19]:
from mpl_toolkits.mplot3d import Axes3D
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111,projection='3d')
ax.scatter(np.log(lengths),np.log(max_growths),np.log(branch_frequ))

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/mpl_toolkits/mplot3d/art3d.py:733: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if zdir == 'x':
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/mpl_toolkits/mplot3d/art3d.py:735: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif zdir == 'y':


In [46]:
from sklearn.cluster import MiniBatchKMeans
kmeans = MiniBatchKMeans(n_clusters=3)
X = np.array([np.log(np.array(lengths)+1),np.log(np.array(max_growths)+1),np.log(np.array(branch_frequ)+1)])
kmeans = kmeans.fit(np.transpose(X))

In [47]:
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111,projection='3d')
ax.scatter(np.log(lengths),np.log(max_growths),kmeans.labels_)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


In [18]:
kmeans.labels_

array([1, 1, 1, ..., 0, 0, 1], dtype=int32)

In [48]:
BAS = [hyph for i,hyph in enumerate(hyph_l) if kmeans.labels_[i]==1]

In [49]:
ambiguous = [hyph for i,hyph in enumerate(hyph_l) if kmeans.labels_[i]==2]

In [55]:
RH = [hyph for i,hyph in enumerate(hyph_l) if kmeans.labels_[i]==0]

In [53]:
len(BAS)

605

In [56]:
plt.close('all')
bas = np.random.choice(RH)
exp.plot([bas.ts[-1]],[[bas.end.label,bas.root.label]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
bas.end.show_source_image(bas.ts[-1],bas.ts[-1])

FileNotFoundError: No such file: '/scratch/shared/AMF914/Fiji.app/20201122_2022_Plate40/Img2/Img_r06_c04.tif'

In [36]:
exp.plot_raw(hyph.ts[-1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [80]:
X = np.array([np.log(lengths),np.log(max_growths),np.log(branch_frequ)])
vectorX = o3d.utility.Vector3dVector(np.transpose(X))
source = o3d.geometry.PointCloud(vectorX)
o3d.visualization.draw_geometries([source])

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


RuntimeError: [1;31m[Open3D ERROR] GLFW Error: X11: The DISPLAY environment variable is missing[0;m

In [41]:
overgrowing = [c for lis in select_hyph.values() for c in lis if c[2]>=20 and c[2]<1500 and c[0]<=20 and c[0]>13]
selected_growth = [c[2] for c in overgrowing]
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.hist(selected_growth,30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([240.,  37.,  32.,  26.,  19.,  25.,  19.,  36.,  34.,  32.,  12.,
          1.,   2.,   0.,   0.,   1.,   0.,   3.,   0.,   1.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.]),
 array([  20.        ,   58.43333333,   96.86666667,  135.3       ,
         173.73333333,  212.16666667,  250.6       ,  289.03333333,
         327.46666667,  365.9       ,  404.33333333,  442.76666667,
         481.2       ,  519.63333333,  558.06666667,  596.5       ,
         634.93333333,  673.36666667,  711.8       ,  750.23333333,
         788.66666667,  827.1       ,  865.53333333,  903.96666667,
         942.4       ,  980.83333333, 1019.26666667, 1057.7       ,
        1096.13333333, 1134.56666667, 1173.        ]),
 <a list of 30 Patch objects>)

In [33]:
overgrowing = [c for c in select_hyph if c[3]>=20]
len_growth = [(c[3],c[0].get_length_um(c[0].ts[-1]),c[0]) for c in overgrowing]

In [ ]:
for c in 

In [34]:
growth = [c[0] for c in len_growth if c[0]<=2000]
length = [c[1] for c in len_growth if c[0]<=2000]
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(growth,np.log(length))

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
overgrowing = [c for c in select_hyph if c[3]>=20]
selected_growth = [c[3] for c in overgrowing]
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.hist(selected_growth,30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([1.247e+03, 4.990e+02, 2.860e+02, 1.880e+02, 1.300e+02, 1.250e+02,
        1.210e+02, 1.140e+02, 1.190e+02, 1.030e+02, 1.050e+02, 8.600e+01,
        8.700e+01, 6.000e+01, 4.300e+01, 3.100e+01, 1.400e+01, 4.000e+00,
        1.000e+00, 0.000e+00, 1.000e+00, 2.000e+00, 0.000e+00, 1.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00]),
 array([ 20.        ,  45.73333333,  71.46666667,  97.2       ,
        122.93333333, 148.66666667, 174.4       , 200.13333333,
        225.86666667, 251.6       , 277.33333333, 303.06666667,
        328.8       , 354.53333333, 380.26666667, 406.        ,
        431.73333333, 457.46666667, 483.2       , 508.93333333,
        534.66666667, 560.4       , 586.13333333, 611.86666667,
        637.6       , 663.33333333, 689.06666667, 714.8       ,
        740.53333333, 766.26666667, 792.        ]),
 <a list of 30 Patch objects>)

In [10]:
plt.close('all')
overgrowing = [c for c in select_hyph if c[3]>=20]
selected_growth = [c[3] for c in overgrowing]
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.hist(selected_growth,30)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([353., 167., 101.,  59.,  31.,  28.,  30.,  31.,  14.,  25.,  26.,
         29.,  43.,  40.,  42.,  33.,   1.,   0.,   0.,   1.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.]),
 array([  20. ,   65.8,  111.6,  157.4,  203.2,  249. ,  294.8,  340.6,
         386.4,  432.2,  478. ,  523.8,  569.6,  615.4,  661.2,  707. ,
         752.8,  798.6,  844.4,  890.2,  936. ,  981.8, 1027.6, 1073.4,
        1119.2, 1165. , 1210.8, 1256.6, 1302.4, 1348.2, 1394. ]),
 <a list of 30 Patch objects>)

In [71]:
hyph,t,tp1,le = choice(overgrowing)
print(le)

45


In [72]:
plt.close('all')
exp.plot([t,tp1],[[hyph.end.label,hyph.root.label]]*2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
hyph.end.show_source_image(t,tp1)

OSError: The file /scratch/shared/AMF914/Fiji.app/20201121_0421_Plate40/Analysis/skeleton_realigned.mat does not exist.

In [5]:
growth_length = [np.sum([len(seg) for seg in segs]) for segs in growth]

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([353., 167., 101.,  59.,  31.,  28.,  30.,  31.,  14.,  25.,  26.,
         29.,  43.,  40.,  42.,  33.,   1.,   0.,   0.,   1.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.]),
 array([  20. ,   65.8,  111.6,  157.4,  203.2,  249. ,  294.8,  340.6,
         386.4,  432.2,  478. ,  523.8,  569.6,  615.4,  661.2,  707. ,
         752.8,  798.6,  844.4,  890.2,  936. ,  981.8, 1027.6, 1073.4,
        1119.2, 1165. , 1210.8, 1256.6, 1302.4, 1348.2, 1394. ]),
 <a list of 30 Patch objects>)

In [15]:
np.mean(branch_frequ)

0.021061224646895957